# Wikipedia training

In this tutorial we will:
 - Learn how to train the NMF topic model on English Wikipedia corpus
 - Compare it with LDA model
 - Evaluate results

In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import json
import logging
import numpy as np
import pandas as pd
import scipy.sparse
import smart_open
import time
from tqdm import tqdm, tqdm_notebook

import gensim.downloader as api
from gensim import matutils
from gensim.corpora import MmCorpus, Dictionary
from gensim.models import LdaModel, CoherenceModel
from gensim.models.nmf import Nmf
from gensim.parsing.preprocessing import preprocess_string

tqdm.pandas()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

# Preprocessing

### Load wikipedia dump
Let's use `gensim.downloader.api` for that

In [2]:
data = api.load("wiki-english-20171001")
article = next(iter(data))

for section_title, section_text in zip(
    article['section_titles'],
    article['section_texts']
):
    print("Section title: %s" % section_title)
    print("Section text: %s" % section_text[:100])

Section title: Introduction
Section text: 




'''Anarchism''' is a political philosophy that advocates self-governed societies based on volun
Section title: Etymology and terminology
Section text: 

The word ''anarchism'' is composed from the word ''anarchy'' and the suffix ''-ism'', themselves d
Section title: History
Section text: 

===Origins===
Woodcut from a Diggers document by William Everard

The earliest anarchist themes ca
Section title: Anarchist schools of thought
Section text: 
Portrait of philosopher Pierre-Joseph Proudhon (1809–1865) by Gustave Courbet. Proudhon was the pri
Section title: Internal issues and debates
Section text: 
consistent with anarchist values is a controversial subject among anarchists.

Anarchism is a philo
Section title: Topics of interest
Section text: Intersecting and overlapping between various schools of thought, certain topics of interest and inte
Section title: Criticisms
Section text: 
Criticisms of anarchism include moral criticisms and pra

Preprocess and save articles

In [3]:
def save_preprocessed_articles(filename, articles):
    with smart_open(filename, 'w+', encoding="utf8") as writer:
        for article in tqdm_notebook(articles):
            article_text = " ".join(
                " ".join(section)
                for section
                in zip(
                    article['section_titles'],
                    article['section_texts']
                )
            )
            article_text = preprocess_string(article_text)

            writer.write(json.dumps(article_text) + '\n')


def get_preprocessed_articles(filename):
    with smart_open(filename, 'r', encoding="utf8") as reader:
        for line in tqdm_notebook(reader):
            yield json.loads(
                line
            )

In [4]:
SAVE_ARTICLES = False

if SAVE_ARTICLES:
    save_preprocessed_articles('wiki_articles.jsonlines', data)

### Create and save dictionary

In [5]:
SAVE_DICTIONARY = False

if SAVE_DICTIONARY:
    dictionary = Dictionary(get_preprocessed_articles('wiki_articles.jsonlines'))
    dictionary.save('wiki.dict')

### Load and filter dictionary

In [6]:
dictionary = Dictionary.load('wiki.dict')
dictionary.filter_extremes()
dictionary.compactify()

2019-01-15 19:31:03,151 : INFO : loading Dictionary object from wiki.dict
2019-01-15 19:31:04,024 : INFO : loaded wiki.dict
2019-01-15 19:31:06,292 : INFO : discarding 1910258 tokens: [('abdelrahim', 49), ('abstention', 120), ('anarcha', 101), ('anarchica', 40), ('anarchosyndicalist', 20), ('antimilitar', 68), ('arbet', 194), ('archo', 100), ('arkhē', 5), ('autonomedia', 118)]...
2019-01-15 19:31:06,293 : INFO : keeping 100000 tokens which were in no less than 5 and no more than 2462447 (=50.0%) documents
2019-01-15 19:31:06,645 : INFO : resulting dictionary: Dictionary(100000 unique tokens: ['abandon', 'abil', 'abl', 'abolit', 'abstent']...)


### MmCorpus wrapper
In this way we'll:

- Make sure that documents are shuffled
- Be able to train-test split corpus without rewriting it

In [7]:
class RandomCorpus(MmCorpus):
    def __init__(self, random_seed=42, testset=False, testsize=1000, *args,
                 **kwargs):
        super().__init__(*args, **kwargs)

        random_state = np.random.RandomState(random_seed)
        self.indices = random_state.permutation(range(self.num_docs))
        if testset:
            self.indices = self.indices[:testsize]
        else:
            self.indices = self.indices[testsize:]

    def __iter__(self):
        for doc_id in self.indices:
            yield self[doc_id]
            
    def __len__(self):
        return len(self.indices)

### Create and save corpus

In [8]:
SAVE_CORPUS = False

if SAVE_CORPUS:
    corpus = (
        dictionary.doc2bow(article)
        for article
        in get_preprocessed_articles('wiki_articles.jsonlines')
    )
    
    RandomCorpus.serialize('wiki.mm', corpus)

### Load train and test corpus
Using `RandomCorpus` wrapper

In [9]:
train_corpus = RandomCorpus(
    random_seed=42, testset=False, testsize=2000, fname='wiki.mm'
)
test_corpus = RandomCorpus(
    random_seed=42, testset=True, testsize=2000, fname='wiki.mm'
)

2019-01-15 19:31:07,323 : INFO : loaded corpus index from wiki.mm.index
2019-01-15 19:31:07,324 : INFO : initializing cython corpus reader from wiki.mm
2019-01-15 19:31:07,325 : INFO : accepted corpus with 4924894 documents, 100000 features, 683375728 non-zero entries
2019-01-15 19:31:08,544 : INFO : loaded corpus index from wiki.mm.index
2019-01-15 19:31:08,544 : INFO : initializing cython corpus reader from wiki.mm
2019-01-15 19:31:08,545 : INFO : accepted corpus with 4924894 documents, 100000 features, 683375728 non-zero entries


## Metrics

In [10]:
def get_execution_time(func):
    start = time.time()

    result = func()

    return (time.time() - start), result


def get_tm_metrics(model, test_corpus):
    W = model.get_topics().T
    H = np.zeros((model.num_topics, len(test_corpus)))
    for bow_id, bow in enumerate(test_corpus):
        for topic_id, word_count in model.get_document_topics(bow):
            H[topic_id, bow_id] = word_count

    pred_factors = W.dot(H)
    pred_factors /= pred_factors.sum(axis=0)
    
    dense_corpus = matutils.corpus2dense(test_corpus, pred_factors.shape[0])

    perplexity = get_tm_perplexity(pred_factors, dense_corpus)

    l2_norm = get_tm_l2_norm(pred_factors, dense_corpus)

    model.normalize = True

    coherence = CoherenceModel(
        model=model,
        corpus=test_corpus,
        coherence='u_mass'
    ).get_coherence()

    topics = model.show_topics()

    model.normalize = False

    return dict(
        perplexity=perplexity,
        coherence=coherence,
        topics=topics,
        l2_norm=l2_norm,
    )


def get_tm_perplexity(pred_factors, dense_corpus):
    return np.exp(-(np.log(pred_factors, where=pred_factors > 0) * dense_corpus).sum() / dense_corpus.sum())


def get_tm_l2_norm(pred_factors, dense_corpus):
    return np.linalg.norm(dense_corpus / dense_corpus.sum(axis=0) - pred_factors)

Define dataframe in which we'll store metrics

In [11]:
tm_metrics = pd.DataFrame()

### Define common params for models

In [12]:
params = dict(
    corpus=train_corpus,
    chunksize=2000,
    num_topics=50,
    id2word=dictionary,
    passes=1,
    eval_every=10,
    minimum_probability=0,
    random_state=42,
)

## Training

### Train NMF and save it
Normalization is turned off to compute metrics correctly

In [13]:
row = dict()
row['model'] = 'nmf'
row['train_time'], nmf = get_execution_time(
    lambda: Nmf(
        use_r=False,
        normalize=False,
        **params
    )
)
nmf.save('nmf.model')

2019-01-15 19:33:21,875 : INFO : Loss (no outliers): 2186.768444126956	Loss (with outliers): 2186.768444126956
2019-01-15 19:34:49,514 : INFO : Loss (no outliers): 2298.434152045061	Loss (with outliers): 2298.434152045061
2019-01-15 19:35:44,330 : INFO : Loss (no outliers): 2101.875741413195	Loss (with outliers): 2101.875741413195
2019-01-15 19:36:46,957 : INFO : Loss (no outliers): 2437.6297291776878	Loss (with outliers): 2437.6297291776878
2019-01-15 19:37:54,953 : INFO : Loss (no outliers): 5519.767671122511	Loss (with outliers): 5519.767671122511
2019-01-15 19:38:49,448 : INFO : Loss (no outliers): 1986.1839688880361	Loss (with outliers): 1986.1839688880361
2019-01-15 19:39:44,683 : INFO : Loss (no outliers): 2870.5244901331134	Loss (with outliers): 2870.5244901331134
2019-01-15 19:40:21,521 : INFO : Loss (no outliers): 1993.3453896080728	Loss (with outliers): 1993.3453896080728
2019-01-15 19:40:53,500 : INFO : Loss (no outliers): 2205.748001914049	Loss (with outliers): 2205.748001

2019-01-15 20:01:45,266 : INFO : Loss (no outliers): 2666.3962088879025	Loss (with outliers): 2666.3962088879025
2019-01-15 20:02:00,497 : INFO : Loss (no outliers): 2165.137588891874	Loss (with outliers): 2165.137588891874
2019-01-15 20:02:16,696 : INFO : Loss (no outliers): 1972.0358673389883	Loss (with outliers): 1972.0358673389883
2019-01-15 20:02:32,239 : INFO : Loss (no outliers): 2152.4675346391805	Loss (with outliers): 2152.4675346391805
2019-01-15 20:02:48,213 : INFO : Loss (no outliers): 2305.3854992392526	Loss (with outliers): 2305.3854992392526
2019-01-15 20:03:03,684 : INFO : Loss (no outliers): 1976.8036563918954	Loss (with outliers): 1976.8036563918954
2019-01-15 20:03:19,195 : INFO : Loss (no outliers): 4285.08225597681	Loss (with outliers): 4285.08225597681
2019-01-15 20:03:34,742 : INFO : Loss (no outliers): 2572.933168490674	Loss (with outliers): 2572.933168490674
2019-01-15 20:03:50,751 : INFO : Loss (no outliers): 3034.3577330312214	Loss (with outliers): 3034.35773

2019-01-15 20:20:36,418 : INFO : Loss (no outliers): 2218.6290864612797	Loss (with outliers): 2218.6290864612797
2019-01-15 20:20:52,090 : INFO : Loss (no outliers): 2067.5063020513276	Loss (with outliers): 2067.5063020513276
2019-01-15 20:21:07,207 : INFO : Loss (no outliers): 2147.2024532449054	Loss (with outliers): 2147.2024532449054
2019-01-15 20:21:22,404 : INFO : Loss (no outliers): 2108.3236711395657	Loss (with outliers): 2108.3236711395657
2019-01-15 20:21:37,427 : INFO : Loss (no outliers): 2203.6092122069194	Loss (with outliers): 2203.6092122069194
2019-01-15 20:21:52,556 : INFO : Loss (no outliers): 2661.3869623246674	Loss (with outliers): 2661.3869623246674
2019-01-15 20:22:07,120 : INFO : Loss (no outliers): 1830.1280505307882	Loss (with outliers): 1830.1280505307882
2019-01-15 20:22:22,658 : INFO : Loss (no outliers): 2108.8037415375165	Loss (with outliers): 2108.8037415375165
2019-01-15 20:22:38,870 : INFO : Loss (no outliers): 2162.9891805304687	Loss (with outliers): 21

2019-01-15 20:38:57,095 : INFO : Loss (no outliers): 3288.809547158024	Loss (with outliers): 3288.809547158024
2019-01-15 20:39:11,926 : INFO : Loss (no outliers): 2176.222127214424	Loss (with outliers): 2176.222127214424
2019-01-15 20:39:26,728 : INFO : Loss (no outliers): 2427.777087295175	Loss (with outliers): 2427.777087295175
2019-01-15 20:39:42,147 : INFO : Loss (no outliers): 2077.3103752487814	Loss (with outliers): 2077.3103752487814
2019-01-15 20:39:55,930 : INFO : Loss (no outliers): 2269.811128442968	Loss (with outliers): 2269.811128442968
2019-01-15 20:40:09,148 : INFO : Loss (no outliers): 2026.9840475664098	Loss (with outliers): 2026.9840475664098
2019-01-15 20:40:22,996 : INFO : Loss (no outliers): 1929.413824110674	Loss (with outliers): 1929.413824110674
2019-01-15 20:40:37,383 : INFO : Loss (no outliers): 2092.8577121320673	Loss (with outliers): 2092.8577121320673
2019-01-15 20:40:51,804 : INFO : Loss (no outliers): 1997.651881209576	Loss (with outliers): 1997.65188120

### Load NMF and store metrics

In [14]:
nmf = Nmf.load('nmf.model')
row.update(get_tm_metrics(nmf, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

nmf.show_topics(50)

2019-01-15 20:44:24,872 : INFO : loading Nmf object from nmf.model
2019-01-15 20:44:25,150 : INFO : loading id2word recursively from nmf.model.id2word.* with mmap=None
2019-01-15 20:44:25,151 : INFO : loaded nmf.model
2019-01-15 20:44:54,148 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2019-01-15 20:44:54,336 : INFO : CorpusAccumulator accumulated stats from 2000 documents


[(0,
  '0.075*"parti" + 0.071*"elect" + 0.042*"democrat" + 0.029*"republican" + 0.022*"vote" + 0.018*"conserv" + 0.017*"liber" + 0.014*"candid" + 0.013*"seat" + 0.013*"labour"'),
 (1,
  '0.039*"book" + 0.038*"centuri" + 0.032*"histori" + 0.032*"languag" + 0.032*"publish" + 0.024*"english" + 0.023*"world" + 0.022*"law" + 0.022*"govern" + 0.021*"nation"'),
 (2,
  '0.050*"war" + 0.036*"forc" + 0.026*"armi" + 0.023*"battl" + 0.021*"attack" + 0.019*"militari" + 0.018*"german" + 0.016*"british" + 0.015*"command" + 0.014*"kill"'),
 (3,
  '0.119*"race" + 0.106*"car" + 0.073*"engin" + 0.035*"model" + 0.030*"driver" + 0.029*"vehicl" + 0.029*"ford" + 0.028*"lap" + 0.023*"electr" + 0.020*"power"'),
 (4,
  '0.102*"leagu" + 0.092*"club" + 0.049*"footbal" + 0.047*"cup" + 0.029*"plai" + 0.028*"season" + 0.028*"divis" + 0.028*"goal" + 0.022*"team" + 0.021*"unit"'),
 (5,
  '0.055*"award" + 0.041*"best" + 0.008*"nomin" + 0.008*"year" + 0.006*"actress" + 0.006*"actor" + 0.005*"perform" + 0.005*"artist" + 

### Train NMF with residuals and save it
Residuals add regularization to the model thus increasing quality, but slows down training

In [15]:
row = dict()
row['model'] = 'nmf_with_r'
row['train_time'], nmf_with_r = get_execution_time(
    lambda: Nmf(
        use_r=True,
        lambda_=200,
        normalize=False,
        **params
    )
)
nmf_with_r.save('nmf_with_r.model')

2019-01-15 20:54:05,363 : INFO : Loss (no outliers): 2179.9524465227146	Loss (with outliers): 2102.354108449905
2019-01-15 20:57:12,821 : INFO : Loss (no outliers): 2268.3200929871823	Loss (with outliers): 2110.928651253909
2019-01-15 20:59:52,789 : INFO : Loss (no outliers): 2085.180592763644	Loss (with outliers): 2041.9026850333144
2019-01-15 21:02:30,113 : INFO : Loss (no outliers): 2425.552273660531	Loss (with outliers): 2299.4821139504706
2019-01-15 21:05:12,256 : INFO : Loss (no outliers): 6084.043369634124	Loss (with outliers): 4478.896745443016
2019-01-15 21:07:48,126 : INFO : Loss (no outliers): 1973.2165098776484	Loss (with outliers): 1962.300562304653
2019-01-15 21:10:15,332 : INFO : Loss (no outliers): 2887.7707165814063	Loss (with outliers): 2253.0207462068242
2019-01-15 21:12:26,921 : INFO : Loss (no outliers): 1968.718319794589	Loss (with outliers): 1958.061798839886
2019-01-15 21:14:36,474 : INFO : Loss (no outliers): 2194.9202571270434	Loss (with outliers): 2068.932739

2019-01-15 23:16:26,633 : INFO : Loss (no outliers): 2101.0331621161545	Loss (with outliers): 2009.8926551115326
2019-01-15 23:18:13,670 : INFO : Loss (no outliers): 2668.4500117317184	Loss (with outliers): 2168.1532193760695
2019-01-15 23:20:04,996 : INFO : Loss (no outliers): 2161.629736053655	Loss (with outliers): 1993.2548324506695
2019-01-15 23:21:52,371 : INFO : Loss (no outliers): 1961.5821979654045	Loss (with outliers): 1830.5366801526282
2019-01-15 23:23:40,125 : INFO : Loss (no outliers): 2146.582950878536	Loss (with outliers): 2084.3624004694666
2019-01-15 23:25:24,220 : INFO : Loss (no outliers): 2296.735495300956	Loss (with outliers): 2070.1072585469383
2019-01-15 23:27:08,597 : INFO : Loss (no outliers): 1962.4322067774845	Loss (with outliers): 1912.7698208747227
2019-01-15 23:28:50,503 : INFO : Loss (no outliers): 4282.5845233371065	Loss (with outliers): 3057.3523936516376
2019-01-15 23:30:33,627 : INFO : Loss (no outliers): 2725.090510533868	Loss (with outliers): 2233.8

2019-01-16 01:23:44,127 : INFO : Loss (no outliers): 2343.9775510877125	Loss (with outliers): 2144.3671549514634
2019-01-16 01:25:20,817 : INFO : Loss (no outliers): 2213.7521980133315	Loss (with outliers): 2068.306787091367
2019-01-16 01:27:03,146 : INFO : Loss (no outliers): 2053.895893791899	Loss (with outliers): 1937.8326485512866
2019-01-16 01:28:44,281 : INFO : Loss (no outliers): 2141.105300358069	Loss (with outliers): 1921.40066058901
2019-01-16 01:30:25,503 : INFO : Loss (no outliers): 2104.2896627284285	Loss (with outliers): 2069.3461308451638
2019-01-16 01:32:00,059 : INFO : Loss (no outliers): 2194.5091361858276	Loss (with outliers): 2101.121907082148
2019-01-16 01:33:38,843 : INFO : Loss (no outliers): 2661.5862423570143	Loss (with outliers): 2314.631507637849
2019-01-16 01:35:17,347 : INFO : Loss (no outliers): 1824.8246722080955	Loss (with outliers): 1787.7885347127317
2019-01-16 01:36:54,564 : INFO : Loss (no outliers): 2111.1179921219923	Loss (with outliers): 1874.3012

2019-01-16 03:26:11,521 : INFO : Loss (no outliers): 2119.101278559414	Loss (with outliers): 2059.4270452072174
2019-01-16 03:27:48,050 : INFO : Loss (no outliers): 3521.6914175427187	Loss (with outliers): 2436.3273863213008
2019-01-16 03:29:26,576 : INFO : Loss (no outliers): 2182.4455446461925	Loss (with outliers): 2149.699829776273
2019-01-16 03:31:02,063 : INFO : Loss (no outliers): 2422.390791633096	Loss (with outliers): 1922.6665499600158
2019-01-16 03:32:43,332 : INFO : Loss (no outliers): 2088.2426906492756	Loss (with outliers): 2004.7046142591066
2019-01-16 03:34:20,250 : INFO : Loss (no outliers): 2272.2733524907258	Loss (with outliers): 2052.6087239672293
2019-01-16 03:35:57,085 : INFO : Loss (no outliers): 2015.1239751640278	Loss (with outliers): 1992.1546281655103
2019-01-16 03:37:36,352 : INFO : Loss (no outliers): 1939.1831252302115	Loss (with outliers): 1931.3598172749896
2019-01-16 03:39:13,136 : INFO : Loss (no outliers): 2121.6048721932993	Loss (with outliers): 2048.

### Load NMF with residuals and store metrics

In [16]:
nmf_with_r = Nmf.load('nmf_with_r.model')
row.update(get_tm_metrics(nmf_with_r, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

nmf_with_r.show_topics(50)

2019-01-16 04:05:48,017 : INFO : loading Nmf object from nmf_with_r.model
2019-01-16 04:05:48,272 : INFO : loading id2word recursively from nmf_with_r.model.id2word.* with mmap=None
2019-01-16 04:05:48,273 : INFO : loading _r from nmf_with_r.model._r.npy with mmap=None
2019-01-16 04:05:48,304 : INFO : loaded nmf_with_r.model
2019-01-16 04:06:27,119 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2019-01-16 04:06:27,253 : INFO : CorpusAccumulator accumulated stats from 2000 documents


[(0,
  '0.062*"parti" + 0.061*"elect" + 0.031*"democrat" + 0.020*"republican" + 0.020*"vote" + 0.013*"liber" + 0.012*"candid" + 0.012*"conserv" + 0.011*"seat" + 0.010*"member"'),
 (1,
  '0.052*"book" + 0.040*"centuri" + 0.039*"publish" + 0.031*"languag" + 0.027*"histori" + 0.025*"work" + 0.023*"english" + 0.022*"king" + 0.019*"polit" + 0.019*"author"'),
 (2,
  '0.031*"armi" + 0.028*"divis" + 0.025*"regiment" + 0.022*"forc" + 0.020*"battalion" + 0.019*"infantri" + 0.019*"command" + 0.017*"brigad" + 0.016*"gener" + 0.012*"corp"'),
 (3,
  '0.110*"race" + 0.059*"car" + 0.033*"engin" + 0.025*"lap" + 0.023*"driver" + 0.021*"ret" + 0.020*"ford" + 0.015*"finish" + 0.015*"motorsport" + 0.015*"chevrolet"'),
 (4,
  '0.130*"club" + 0.068*"cup" + 0.046*"footbal" + 0.044*"goal" + 0.032*"leagu" + 0.031*"unit" + 0.031*"plai" + 0.030*"match" + 0.026*"score" + 0.021*"player"'),
 (5,
  '0.041*"award" + 0.030*"best" + 0.006*"nomin" + 0.005*"actress" + 0.005*"year" + 0.004*"actor" + 0.004*"won" + 0.004*"pe

### Train LDA and save it
That's a common model to do Topic Modeling

In [17]:
row = dict()
row['model'] = 'lda'
row['train_time'], lda = get_execution_time(
    lambda: LdaModel(**params)
)
lda.save('lda.model')

2019-01-16 04:06:27,576 : INFO : using symmetric alpha at 0.02
2019-01-16 04:06:27,576 : INFO : using symmetric eta at 0.02
2019-01-16 04:06:27,589 : INFO : using serial LDA version on this node
2019-01-16 04:06:28,185 : INFO : running online (single-pass) LDA training, 50 topics, 1 passes over the supplied corpus of 4922894 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-01-16 04:06:28,910 : INFO : PROGRESS: pass 0, at document #2000/4922894
2019-01-16 04:06:31,888 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-16 04:06:32,429 : INFO : topic #36 (0.020): 0.006*"new" + 0.004*"year" + 0.004*"work" + 0.004*"hous" + 0.004*"railwai" + 0.003*"american" + 0.003*"state" + 0.003*"includ" + 0.003*"game" + 0.003*"senat"
2019-01-16 04:06:32,432 : INFO : topic #19 (0.020): 0.005*"new" + 0.004*"nation" + 0.004*"includ" + 0.003*"time" + 0.003*"year" + 

2019-01-16 04:06:52,113 : INFO : topic diff=0.225180, rho=0.408248
2019-01-16 04:06:52,836 : INFO : PROGRESS: pass 0, at document #14000/4922894
2019-01-16 04:06:55,314 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-16 04:06:55,847 : INFO : topic #39 (0.020): 0.011*"servic" + 0.011*"unit" + 0.010*"air" + 0.009*"presid" + 0.008*"state" + 0.007*"univers" + 0.007*"nation" + 0.007*"war" + 0.006*"militari" + 0.006*"serv"
2019-01-16 04:06:55,849 : INFO : topic #0 (0.020): 0.019*"war" + 0.017*"armi" + 0.015*"battalion" + 0.013*"forc" + 0.012*"battl" + 0.011*"divis" + 0.010*"command" + 0.008*"attack" + 0.007*"regiment" + 0.007*"brigad"
2019-01-16 04:06:55,850 : INFO : topic #13 (0.020): 0.057*"state" + 0.044*"univers" + 0.023*"unit" + 0.018*"colleg" + 0.014*"texa" + 0.010*"new" + 0.009*"carolina" + 0.008*"activ" + 0.008*"inact" + 0.008*"california"
2019-01-16 04:06:55,852 : INFO : topic #45 (0.020): 0.018*"art" + 0.013*"london" + 0.011*"john" + 0.011*"bo

2019-01-16 04:07:21,980 : INFO : topic #10 (0.020): 0.015*"engin" + 0.009*"cell" + 0.009*"vehicl" + 0.009*"car" + 0.007*"product" + 0.007*"model" + 0.007*"type" + 0.006*"acid" + 0.006*"produc" + 0.006*"electr"
2019-01-16 04:07:21,982 : INFO : topic #18 (0.020): 0.007*"surfac" + 0.005*"magnet" + 0.005*"form" + 0.005*"imag" + 0.005*"light" + 0.005*"resist" + 0.005*"high" + 0.005*"wave" + 0.005*"effect" + 0.005*"materi"
2019-01-16 04:07:21,984 : INFO : topic #0 (0.020): 0.025*"armi" + 0.024*"war" + 0.016*"regiment" + 0.016*"forc" + 0.013*"battl" + 0.013*"command" + 0.013*"divis" + 0.010*"attack" + 0.009*"captain" + 0.009*"lieuten"
2019-01-16 04:07:21,986 : INFO : topic #37 (0.020): 0.011*"releas" + 0.009*"love" + 0.009*"song" + 0.006*"appear" + 0.006*"man" + 0.005*"time" + 0.005*"like" + 0.005*"live" + 0.004*"video" + 0.004*"friend"
2019-01-16 04:07:21,988 : INFO : topic #35 (0.020): 0.036*"minist" + 0.020*"prime" + 0.006*"born" + 0.006*"announc" + 0.006*"indonesia" + 0.005*"seri" + 0.005

2019-01-16 04:07:43,009 : INFO : topic #25 (0.020): 0.022*"draw" + 0.021*"clai" + 0.020*"final" + 0.017*"def" + 0.017*"tournament" + 0.016*"open" + 0.015*"hard" + 0.014*"doubl" + 0.012*"titl" + 0.011*"runner"
2019-01-16 04:07:43,011 : INFO : topic #27 (0.020): 0.025*"german" + 0.025*"russian" + 0.019*"germani" + 0.015*"russia" + 0.014*"soviet" + 0.014*"jewish" + 0.013*"moscow" + 0.012*"israel" + 0.009*"polish" + 0.008*"rabbi"
2019-01-16 04:07:43,012 : INFO : topic #48 (0.020): 0.061*"octob" + 0.058*"januari" + 0.054*"march" + 0.052*"septemb" + 0.051*"novemb" + 0.049*"april" + 0.048*"august" + 0.047*"june" + 0.046*"decemb" + 0.045*"juli"
2019-01-16 04:07:43,018 : INFO : topic diff=0.293284, rho=0.229416
2019-01-16 04:07:48,367 : INFO : -11.688 per-word bound, 3299.5 perplexity estimate based on a held-out corpus of 2000 documents with 564343 words
2019-01-16 04:07:48,368 : INFO : PROGRESS: pass 0, at document #40000/4922894
2019-01-16 04:07:50,697 : INFO : merging changes from 2000 docu

2019-01-16 04:08:08,713 : INFO : topic #21 (0.020): 0.008*"word" + 0.006*"languag" + 0.006*"english" + 0.006*"god" + 0.005*"centuri" + 0.005*"form" + 0.005*"christian" + 0.005*"tradit" + 0.005*"person" + 0.005*"peopl"
2019-01-16 04:08:08,715 : INFO : topic #49 (0.020): 0.035*"swedish" + 0.027*"poland" + 0.025*"sweden" + 0.018*"east" + 0.016*"alt" + 0.015*"gmina" + 0.014*"voivodeship" + 0.013*"approxim" + 0.012*"greater" + 0.012*"li"
2019-01-16 04:08:08,721 : INFO : topic diff=0.245166, rho=0.200000
2019-01-16 04:08:09,411 : INFO : PROGRESS: pass 0, at document #52000/4922894
2019-01-16 04:08:11,688 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-16 04:08:12,236 : INFO : topic #21 (0.020): 0.007*"word" + 0.006*"languag" + 0.005*"centuri" + 0.005*"person" + 0.005*"english" + 0.005*"god" + 0.005*"form" + 0.005*"tradit" + 0.005*"christian" + 0.005*"peopl"
2019-01-16 04:08:12,238 : INFO : topic #25 (0.020): 0.021*"final" + 0.018*"tournament" + 0.017*"r

...

2019-01-16 06:22:52,298 : INFO : topic diff=0.003291, rho=0.020269
2019-01-16 06:22:52,569 : INFO : PROGRESS: pass 0, at document #4870000/4922894
2019-01-16 06:22:54,569 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-16 06:22:55,131 : INFO : topic #28 (0.020): 0.029*"build" + 0.024*"hous" + 0.014*"built" + 0.012*"locat" + 0.012*"street" + 0.012*"site" + 0.011*"histor" + 0.009*"park" + 0.009*"citi" + 0.009*"place"
2019-01-16 06:22:55,132 : INFO : topic #26 (0.020): 0.032*"world" + 0.031*"women" + 0.028*"championship" + 0.026*"olymp" + 0.022*"event" + 0.022*"men" + 0.022*"medal" + 0.019*"athlet" + 0.018*"japan" + 0.017*"time"
2019-01-16 06:22:55,133 : INFO : topic #18 (0.020): 0.010*"water" + 0.007*"light" + 0.007*"high" + 0.007*"energi" + 0.006*"surfac" + 0.006*"earth" + 0.006*"time" + 0.005*"effect" + 0.005*"temperatur" + 0.005*"materi"
2019-01-16 06:22:55,135 : INFO : topic #17 (0.020): 0.060*"race" + 0.020*"car" + 0.016*"team" + 0.013*"finish"

2019-01-16 06:23:16,600 : INFO : topic #6 (0.020): 0.060*"music" + 0.030*"perform" + 0.019*"theatr" + 0.018*"compos" + 0.016*"plai" + 0.015*"festiv" + 0.014*"danc" + 0.013*"orchestra" + 0.012*"opera" + 0.011*"piano"
2019-01-16 06:23:16,601 : INFO : topic #10 (0.020): 0.019*"engin" + 0.012*"power" + 0.012*"product" + 0.011*"design" + 0.009*"produc" + 0.009*"model" + 0.008*"us" + 0.008*"electr" + 0.008*"type" + 0.007*"vehicl"
2019-01-16 06:23:16,603 : INFO : topic #4 (0.020): 0.136*"school" + 0.039*"colleg" + 0.039*"student" + 0.033*"univers" + 0.030*"high" + 0.028*"educ" + 0.016*"year" + 0.011*"graduat" + 0.010*"state" + 0.009*"district"
2019-01-16 06:23:16,604 : INFO : topic #32 (0.020): 0.026*"speci" + 0.011*"famili" + 0.010*"plant" + 0.008*"white" + 0.008*"genu" + 0.008*"bird" + 0.007*"forest" + 0.007*"fish" + 0.007*"red" + 0.007*"tree"
2019-01-16 06:23:16,605 : INFO : topic #26 (0.020): 0.032*"world" + 0.030*"women" + 0.028*"championship" + 0.026*"olymp" + 0.023*"men" + 0.022*"event

2019-01-16 06:23:34,084 : INFO : topic #40 (0.020): 0.051*"bar" + 0.039*"africa" + 0.033*"african" + 0.031*"text" + 0.028*"till" + 0.027*"south" + 0.026*"color" + 0.023*"black" + 0.013*"tropic" + 0.013*"storm"
2019-01-16 06:23:34,085 : INFO : topic #48 (0.020): 0.073*"octob" + 0.071*"march" + 0.071*"septemb" + 0.063*"novemb" + 0.063*"januari" + 0.062*"juli" + 0.062*"august" + 0.061*"april" + 0.061*"decemb" + 0.059*"june"
2019-01-16 06:23:34,086 : INFO : topic #31 (0.020): 0.068*"australia" + 0.056*"australian" + 0.051*"new" + 0.040*"china" + 0.033*"zealand" + 0.033*"south" + 0.026*"chines" + 0.021*"sydnei" + 0.015*"melbourn" + 0.014*"queensland"
2019-01-16 06:23:34,092 : INFO : topic diff=0.003495, rho=0.020215
2019-01-16 06:23:34,363 : INFO : PROGRESS: pass 0, at document #4896000/4922894
2019-01-16 06:23:36,376 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-16 06:23:36,934 : INFO : topic #25 (0.020): 0.045*"round" + 0.044*"final" + 0.027*"tourn

2019-01-16 06:23:55,548 : INFO : topic #42 (0.020): 0.026*"king" + 0.019*"centuri" + 0.010*"princ" + 0.009*"empir" + 0.009*"emperor" + 0.009*"kingdom" + 0.009*"greek" + 0.008*"roman" + 0.007*"ancient" + 0.006*"templ"
2019-01-16 06:23:55,549 : INFO : topic #16 (0.020): 0.031*"church" + 0.017*"famili" + 0.017*"di" + 0.017*"son" + 0.015*"marri" + 0.014*"year" + 0.013*"father" + 0.013*"life" + 0.013*"born" + 0.012*"daughter"
2019-01-16 06:23:55,555 : INFO : topic diff=0.003067, rho=0.020191
2019-01-16 06:23:55,828 : INFO : PROGRESS: pass 0, at document #4908000/4922894
2019-01-16 06:23:57,819 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-16 06:23:58,383 : INFO : topic #43 (0.020): 0.033*"san" + 0.022*"spanish" + 0.017*"mexico" + 0.016*"del" + 0.014*"spain" + 0.012*"santa" + 0.011*"brazil" + 0.011*"juan" + 0.010*"josé" + 0.010*"francisco"
2019-01-16 06:23:58,385 : INFO : topic #32 (0.020): 0.026*"speci" + 0.011*"famili" + 0.009*"plant" + 0.008*"white

2019-01-16 06:24:12,492 : INFO : topic diff=0.003718, rho=0.020166
2019-01-16 06:24:17,164 : INFO : -11.794 per-word bound, 3550.7 perplexity estimate based on a held-out corpus of 2000 documents with 582002 words
2019-01-16 06:24:17,165 : INFO : PROGRESS: pass 0, at document #4920000/4922894
2019-01-16 06:24:19,188 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-16 06:24:19,745 : INFO : topic #27 (0.020): 0.055*"born" + 0.034*"russian" + 0.025*"american" + 0.021*"russia" + 0.020*"soviet" + 0.017*"polish" + 0.015*"jewish" + 0.014*"poland" + 0.014*"republ" + 0.013*"moscow"
2019-01-16 06:24:19,746 : INFO : topic #48 (0.020): 0.071*"octob" + 0.070*"septemb" + 0.069*"march" + 0.062*"decemb" + 0.062*"januari" + 0.062*"novemb" + 0.062*"juli" + 0.061*"august" + 0.060*"april" + 0.058*"june"
2019-01-16 06:24:19,748 : INFO : topic #1 (0.020): 0.038*"album" + 0.028*"song" + 0.026*"releas" + 0.026*"record" + 0.021*"band" + 0.016*"singl" + 0.015*"music" + 0.01

### Load LDA and store metrics

In [18]:
lda = LdaModel.load('lda.model')
row.update(get_tm_metrics(lda, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

lda.show_topics(50)

2019-01-16 06:24:27,064 : INFO : loading LdaModel object from lda.model
2019-01-16 06:24:27,070 : INFO : loading expElogbeta from lda.model.expElogbeta.npy with mmap=None
2019-01-16 06:24:27,077 : INFO : setting ignored attribute dispatcher to None
2019-01-16 06:24:27,078 : INFO : setting ignored attribute id2word to None
2019-01-16 06:24:27,078 : INFO : setting ignored attribute state to None
2019-01-16 06:24:27,079 : INFO : loaded lda.model
2019-01-16 06:24:27,079 : INFO : loading LdaState object from lda.model.state
2019-01-16 06:24:27,173 : INFO : loaded lda.model.state
2019-01-16 06:24:41,257 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2019-01-16 06:24:41,452 : INFO : CorpusAccumulator accumulated stats from 2000 documents


[(0,
  '0.033*"war" + 0.028*"armi" + 0.021*"forc" + 0.020*"command" + 0.015*"militari" + 0.015*"battl" + 0.013*"gener" + 0.012*"offic" + 0.011*"divis" + 0.011*"regiment"'),
 (1,
  '0.038*"album" + 0.028*"song" + 0.026*"releas" + 0.026*"record" + 0.021*"band" + 0.016*"singl" + 0.015*"music" + 0.014*"chart" + 0.013*"track" + 0.010*"guitar"'),
 (2,
  '0.062*"german" + 0.039*"germani" + 0.025*"van" + 0.023*"von" + 0.020*"der" + 0.019*"dutch" + 0.019*"berlin" + 0.015*"swedish" + 0.014*"netherland" + 0.014*"sweden"'),
 (3,
  '0.032*"john" + 0.027*"william" + 0.019*"british" + 0.015*"georg" + 0.015*"london" + 0.014*"thoma" + 0.014*"sir" + 0.014*"jame" + 0.013*"royal" + 0.013*"henri"'),
 (4,
  '0.137*"school" + 0.040*"colleg" + 0.039*"student" + 0.033*"univers" + 0.030*"high" + 0.028*"educ" + 0.016*"year" + 0.011*"graduat" + 0.010*"state" + 0.009*"campu"'),
 (5,
  '0.030*"game" + 0.009*"develop" + 0.009*"player" + 0.008*"releas" + 0.008*"us" + 0.008*"softwar" + 0.008*"version" + 0.008*"user" +

## Results

In [19]:
tm_metrics

,coherence,l2_norm,model,perplexity,topics,train_time
0,-2.814135,7.265412,nmf,975.740399,"[(24, 0.131*""mount"" + 0.129*""lemmon"" + 0.129*""...",4394.560518
1,-2.436650,7.268837,nmf_with_r,985.570926,"[(49, 0.112*""peak"" + 0.111*""kitt"" + 0.111*""mou...",26451.927848
2,-2.514469,7.371544,lda,4727.075546,"[(35, 0.034*""kong"" + 0.034*""japanes"" + 0.033*""...",8278.891060


#### RAM Usage:
- nmf: 100-150Mb
- nmf_with_r: 3-9Gb
- lda: 100Mb

In [20]:
for row_idx, row in tm_metrics.iterrows():
    print('='*20)
    print(row['model'])
    print('='*20)
    print()
    print("\n\n".join(str(topic) for topic in row['topics']))
    print('\n')

nmf

(24, '0.131*"mount" + 0.129*"lemmon" + 0.129*"peak" + 0.127*"kitt" + 0.127*"spacewatch" + 0.065*"survei" + 0.037*"octob" + 0.031*"septemb" + 0.023*"css" + 0.023*"catalina"')

(32, '0.196*"linear" + 0.195*"socorro" + 0.045*"septemb" + 0.039*"neat" + 0.035*"palomar" + 0.032*"octob" + 0.024*"kitt" + 0.024*"peak" + 0.024*"spacewatch" + 0.023*"anderson"')

(8, '0.331*"align" + 0.270*"left" + 0.071*"right" + 0.040*"text" + 0.035*"style" + 0.022*"center" + 0.013*"bar" + 0.009*"till" + 0.008*"bgcolor" + 0.008*"color"')

(27, '0.186*"district" + 0.027*"pennsylvania" + 0.022*"grade" + 0.017*"fund" + 0.017*"educ" + 0.017*"basic" + 0.016*"level" + 0.014*"oblast" + 0.014*"rural" + 0.013*"tax"')

(48, '0.103*"art" + 0.066*"museum" + 0.040*"paint" + 0.035*"work" + 0.026*"artist" + 0.024*"galleri" + 0.022*"exhibit" + 0.019*"collect" + 0.015*"histori" + 0.013*"jpg"')

(11, '0.122*"new" + 0.043*"york" + 0.009*"zealand" + 0.007*"jersei" + 0.006*"american" + 0.006*"time" + 0.006*"australia" + 0.005*"

As we can see, NMF can be significantly faster than LDA without sacrificing quality of topics too much (or not sacrificing at all)

Moreover, NMF can be very flexible on RAM usage due to sparsity option, which leaves only small amount of elements in inner matrices.